In [21]:
import pandas as pd

In [22]:
data_train = pd.read_csv("dataset.tsv", delimiter="\t", header=None)
data_test = pd.read_csv("data_test.tsv", delimiter="\t", header=None)
most_word = []
with open("most_word.txt", "r") as file:
       for line in file.readlines():
            temp = line.replace('\x00', '')
            most_word.append(temp.rstrip('\n'))


In [23]:
most_w =  []
for i in most_word:
    if i != '':
        most_w.append(i)
most_w

['ÿþsebagai',
 'saya',
 'nya',
 'bahwa',
 'dia',
 'adalah',
 'untuk',
 'pada',
 'adalah',
 'dengan',
 'mereka',
 'menjadi',
 'di',
 'satu',
 'memiliki',
 'ini',
 'dari',
 'oleh',
 'hot',
 'kata',
 'tapi',
 'apa',
 'beberapa',
 'adalah',
 'itu',
 'anda',
 'atau',
 'memiliki',
 'itu',
 'dari',
 'untuk',
 'dan',
 'sebuah',
 'di',
 'kami',
 'bisa',
 'out',
 'lainnya',
 'yang',
 'yang',
 'melakukan',
 'mereka',
 'waktu',
 'jika',
 'akan',
 'bagaimana',
 'kata',
 'an',
 'masing-masing',
 'memberitahu',
 'tidak',
 'Kumpulan',
 'tiga',
 'ingin',
 'udara',
 'baik',
 'juga',
 'bermain',
 'kecil',
 'end',
 'menempatkan',
 'rumah',
 'baca',
 'tangan',
 'pelabuhan',
 'besar',
 'mantra',
 'tambahkan',
 'bahkan',
 'tanah',
 'di sini',
 'harus',
 'besar',
 'tinggi',
 'seperti',
 'ikuti',
 'tindakan',
 'mengapa',
 'bertanya',
 'laki-laki',
 'perubahan',
 'pergi',
 'cahaya',
 'jenis',
 'off',
 'perlu',
 'rumah',
 'gambar',
 'coba',
 'kami',
 'lagi',
 'hewan',
 'titik',
 'ibu',
 'dunia',
 'dekat',
 'memb

In [24]:
world_location = pd.read_csv("worldcities.csv")

In [25]:
def isLocation(tokens, i, tag):
    word = tokens[i]
    if word.lower() in world_location['city'].str.lower().values or word.lower() in world_location['country'].str.lower().values:
        return True
    if tag != 'NNP' and tag != 'NN':
        return False
    if not (word[0].isupper() and not word.lower() in most_w):
        return False
    if (tokens[i-1] == 'di' or tokens[i-1] == ','):
        return True
    if tokens[i+1].lower() == 'tenggara':
        return True

In [26]:
def isPerson(word, tag):
    if tag != 'NNP':
        return False
    return (word[0].isupper() and not word.lower() in most_w)

In [27]:
def isOrganization(word, tag):
    if tag != 'NNP' and tag != 'NN':
        return False
    return ((word.isupper) or (word[0].isupper())) and not word.lower() in most_w

In [28]:
train_set = list(data_train.iterrows())
tokens = []
postags = []
NER = []
correct = 0
total = 0
for i, row in enumerate(train_set):
    word = row[1][0]
    tag = row[1][1]
    label = row[1][2]
    if not row[1].isnull()[0]:
        tokens.append(word)
        postags.append(tag)
        NER.append(label)
    else:
        if len(tokens) == 0:
            continue
        print(tokens)
        print(postags)

        NER_label = []
        for i in range(len(tokens)):
            # Check Location
            if isLocation(tokens, i, postags[i]):
                if len(NER_label) > 0 and NER_label[-1] == 'B-LOC':
                    NER_label.append('I-LOC')
                else:
                    NER_label.append('B-LOC')
            # Check Person
            elif isPerson(tokens[i], postags[i]):
                if len(NER_label) > 0 and NER_label[-1] == 'B-PER':
                    NER_label.append('I-PER')
                else:
                    NER_label.append('B-PER')
            # Check Organization
            elif isOrganization(tokens[i], postags[i]):
                if len(NER_label) > 0 and NER_label[-1] == 'B-ORG':
                    NER_label.append('I-ORG')
                else:
                    NER_label.append('B-ORG')
            else:
                NER_label.append('O')
        print(NER_label)
        tokens = []
        postags = []
        for ner, y in zip(NER_label, NER):
            total += 1
            if ner == y:
                correct += 1
        NER = []
    

['SEA', 'GAMES', '2019', 'merupakan', 'sebuah', 'Pesta', 'Olahraga', 'Asia', 'Tenggara', '2019', 'yang', 'akan', 'diselenggarakan', 'di', 'Manila', ',', 'ibu', 'kota', 'dari', 'negara', 'Filipina', '.']
['NN', 'NN', 'CDP', 'VBT', 'NN', 'NN', 'NN', 'NNP', 'NN', 'NN', 'SC', 'MD', 'VBT', 'IN', 'NNP', ',', 'NNP', 'NN', 'IN', 'NN', 'NN', '.']
['B-ORG', 'I-ORG', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'B-LOC', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'B-LOC', 'O', 'O', 'B-LOC', 'O']
['Pesta', 'Olahraga', 'Asia', 'Tenggara', 'kali', 'ini', 'akan', 'menjadi', 'Pesta', 'Olahraga', 'Asia', 'Tenggara', 'yang', 'ke', '-', '30', '(', 'tiga', 'puluh)', '.']
['NN', 'NN', 'NNP', 'NN', 'NN', 'DT', 'MD', 'VBT', 'NN', 'NN', 'NNP', 'NN', 'SC', 'IN', '-', 'CDP', 'OP', 'CDP', 'CDP', '.']
['B-ORG', 'I-ORG', 'B-LOC', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'B-LOC', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['Pesta', 'Olahraga', 'Asia', 'Tenggara', 'ini', 'akan', 'diselenggarakan', '

In [29]:
print('Akurasi:', (correct/total)*100)

Akurasi: 56.95708712613784
